In [3]:
using Coarsening, Graphs, LinearOrdering, MatrixMarket

In [4]:
include("./dynamictreewidth.jl")

makeadj (generic function with 1 method)

In [5]:
fname = "./MISKnowledgeMap_CiteCnt.mtx"

"./MISKnowledgeMap_CiteCnt.mtx"

In [6]:
adj = makeadj(mmread(fname))

LoadError: BoundsError: attempt to access 1-element Vector{Int64} at index [2]